In [4]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores.pinecone import Pinecone
from domino_data.vectordb import DominoPineconeConfiguration
from mlflow.deployments import get_deploy_client
from langchain_community.embeddings import MlflowEmbeddings
from langchain_community.chat_models import ChatMlflow

import pinecone
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [5]:
#os.environ['ANTHROPIC_API_KEY'] = getpass("Enter Anthropic key:")
#os.environ['OPENAI_API_KEY'] = getpass("Enter OpenAI API key:")
#os.environ['PINECONE_API_KEY'] = getpass("Enter Pinecone API key:")
#os.environ['PINECONE_ENV'] = getpass("Enter Pinecone Environment:")

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') 
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_API_ENV')
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [6]:
# Create embeddings to embed queries
embed = MlflowEmbeddings(
    target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
    endpoint="embedding-ada-002ja",
)

In [7]:
text_field = "text"

datasource_name = "nwh-benefits"
#print(PINECONE_ENV)
#Vector Access 
conf = DominoPineconeConfiguration(datasource=datasource_name)

#print(conf)

api_key = os.environ.get("DOMINO_VECTOR_DB_METADATA", datasource_name)
#api_key = PINECONE_API_KEY

#print(api_key)

# initialize pinecone
pinecone.init(
    api_key=api_key,
    environment=PINECONE_ENV,
    openapi_config=conf
)

index_name = "nwh-plus-benefits"
index = pinecone.Index(index_name)

# switch back to normal index for langchain
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [8]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00318,
 'namespaces': {'': {'vector_count': 318}},
 'total_vector_count': 318}

In [9]:
prompt_template = """You are an AI assistant for answering questions about employee benefits in health care policy documentation.
You are given the following extracted parts of a long document and a question. 
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
If the question is not about employee benefits or policy coverage, politely inform them that you are tuned to only answer questions pertaining to policy coverage.
Question: {question}
=========
{context}
=========
Answer in Markdown:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["question", "context"])
#
chain_type_kwargs = {"prompt": PROMPT}

In [10]:
rag_llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-4',
    temperature=0.0
)

In [26]:
# Using the AI Gateway Endpoint
rag_llm = ChatMlflow(
        target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
        endpoint="chat-gpt35turbo-ja"
    )

In [11]:
qa_chain = RetrievalQA.from_chain_type(llm=rag_llm,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

In [14]:
user_question = input("Please ask your policy question:")
result = qa_chain(user_question)

Please ask your policy question: What about hearing?


In [15]:
result['result']

'Northwind Health Plus, the health care provider for Contoso employees, offers comprehensive hearing care coverage. This includes hearing tests and evaluations, hearing aids and associated services, and hearing aid fittings and adjustments. However, the plan does not cover the cost of custom ear molds for hearing aids or over-the-counter hearing aids. The coverage is limited to individuals who are 18 years or older.\n\nFor hearing aid coverage, Northwind Health Plus covers up to $1,500 every 3 years for all hearing aid services, including the hearing aid itself. Any additional costs over this amount are the responsibility of the employee. \n\nTo take advantage of this coverage, employees must receive care from an in-network provider. Employees are also advised to keep all receipts and submit them to Northwind Health Plus for reimbursement. If there are any questions about the coverage, employees can call Northwind Health Plus customer service.'

In [16]:
result['source_documents'][0].page_content

'It is important to note that Northwind Health Plus does not cover the cost of custom ear \nmolds for hearing aids, nor does the plan cover any over -the-counter hearing aids or other \ndevices. Additionally, hearing care coverage is limited to individuals 18 years of age or \nolder.  \nAt Contoso, we also want to make sure that our e mployees have the best hearing care \npossible. Here are a few tips to help our employees make the most of their coverage:  \n• Schedule regular hearing tests and evaluations. This can help you stay on top of your \nhearing health and detect any issues early.  \n• Try to get all of your hearing care needs met by the same provider. This can help you \nestablish a relationship with the provider and make it easier to get the care you need.  \n• Make sure to keep all of your receipts, and submit them to Northwind Health Plus  for \nreimbursement.  \n• Ask your provider about any discounts or promotions they may have available.'